# Simulation and Inference for Neuroscience
## Exercises Week 3: Parameter inference for simulators in neuroscience

The notebook below contains the exercises for week 3.
The exercises are designed to follow the lecture material and do not need deep knowledge of python to be solved.

There are two types of exercises:
- **Tasks**: Where you need to write code, i.e. "implement a function that does X".
- **Questions**: Where you need to answer questions using the code you wrote. For example "vary the inputs and interpret the results", will require you to provide one or more plots together with your answer. To answer the questions you can use Markdown cells.

All necessary dependencies are imported in the first cell and you should not need to import anything else. In addition, we also provide some helper functions that can be used to help you answering the tasks. When solving the tasks, feel free to change the provided function signatures and code as you see fit. However, we recommend sticking with the provided signatures as they reflect common conventions and will make it easier to add additional functionality during the exercise. Type hints are provided for the function signatures and the arguments and each task should only require a few lines of code.

In [14]:
# configure jax to use 64bit precision and cpu
from jax import config

config.update("jax_enable_x64", True)
config.update("jax_platform_name", "cpu")

import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".8"

In [15]:
# add the project root to the python path
import sys
sys.path.append("..")

# import the necessary dependencies
import matplotlib.pyplot as plt
import numpy as np
import optax
import torch
import jax
from jax import numpy as jnp
from jax import jit, vmap, value_and_grad

# jaxley imports
import jaxley as jx
from jaxley.channels import HH
from jaxley.stimulus import step_current
from jaxley.optimize.utils import l2_norm
from jaxley.optimize.transforms import SigmoidTransform, ParamTransform

# sbi imports
from sbi.utils import BoxUniform
from sbi.inference import SNPE
from sbi.analysis import pairplot

# Typing imports
from typing import Callable,Union, Optional, Tuple

In [16]:
# utils
def plot_observation(
    t_obs: Union[jnp.ndarray, torch.tensor],
    v_obs: Union[jnp.ndarray, torch.tensor],
    i_obs: Optional[Union[jnp.ndarray, torch.tensor]] = None,
) -> Tuple[plt.Figure, plt.Axes]:
    """Plot the observed voltage and stimulus.

    If stimulus is provided it will create a figure with 2 axes, otherwise it will create a figure with 1 axis.

    Args:
        t_obs: The time points of the observed voltage and stimulus (ms).
        v_obs: The observed voltage (mV).
        i_obs: The observed stimulus (nA).

    Returns:
        fig: The figure.
        ax: The axes.
    """
    num_rows = 2 if i_obs is not None else 1
    height_ratios = [3, 1] if i_obs is not None else [1]
    fig, ax = plt.subplots(
        num_rows,
        1,
        figsize=(10, 5),
        sharex=True,
        gridspec_kw={"height_ratios": height_ratios},
    )
    if num_rows == 1:
        ax = [ax]
    ax[0].plot(t_obs, v_obs, c="k")
    ax[0].set_xlabel("t (ms)")
    ax[0].set_ylabel("V (mV)")
    if i_obs is not None:
        ax[1].plot(t_obs, i_obs, c="k")
        ax[1].set_xlabel("t (ms)")
        ax[1].set_ylabel("stimulus (nA)")
    return fig, ax


def make_plot(
    t_est: Union[jnp.ndarray, torch.tensor],
    v_est: Union[jnp.ndarray, torch.tensor],
    t_obs: Optional[Union[jnp.ndarray, torch.tensor]] = None,
    v_obs: Optional[Union[jnp.ndarray, torch.tensor]] = None,
    i_obs: Optional[Union[jnp.ndarray, torch.tensor]] = None,
) -> Tuple[plt.Figure, plt.Axes]:
    """Plot the estimated voltage and stimulus (along with the observed data if provided).

    Args:
        t_est: The time points of the estimated voltage (ms).
        v_est: The estimated voltage (mV).
        t_obs: The time points of the observed voltage (ms).
        v_obs: The observed voltage (mV).
        i_obs: The observed stimulus (nA).
    """
    if t_obs is not None:
        fig, ax = plot_observation(t_obs, v_obs, i_obs)
    else:
        fig, ax = plt.subplots(1, 1, figsize=(10, 5))
        ax = [ax]
    ax[0].plot(t_est, v_est, c="r", alpha=0.5)
    ax[0].set_xlabel("t (ms)")
    ax[0].set_ylabel("V (mV)")
    return fig, ax

# Infer parameters of a single compartment HH model

In week 1 we have learned how to simulate neural dynamics using the Hodgkin-Huxley model, i.e. going from parameters to observations. We call this the forward problem. 
In week 2 on the otherhand, you were introduced to methods that solve the so called inverse problem, i.e. going from observations to parameters. 
Now we will put these two together and see how we can combine simulation and inference methods to infer the parameters of a Hodgkin-Huxley model from a measured voltage trace.

Lets start by importing and looking at the data, before attempting to infer the model parameters at different levels of sophistication.

In [ ]:
# load observation (note that we convert the data to a jax array)
t_obs, v_obs, stim_obs = jnp.array(np.loadtxt("../data/observation.csv", unpack=True))

# stimulus information
sampling_interval = t_obs[1] - t_obs[0]
stim_amp = jnp.max(stim_obs)
stim_onset = t_obs[stim_obs.nonzero()][0]
stim_end = t_obs[stim_obs.nonzero()][-1] + sampling_interval
stim_dur = stim_end - stim_onset
t_max = t_obs[-1] + sampling_interval

print("Sampling interval: ", sampling_interval, "ms")
print("Max. time: ", t_max, "ms")
print("Stimulus amplitude: ", stim_amp, "nA")
print("Stimulus (end | duration | onset): ", f"{stim_end:.2f} | {stim_dur:.2f} | {stim_onset:.2f} ms")

# plot the observation
plot_observation(t_obs, v_obs, i_obs=stim_obs)
plt.show()

## Hand-tuning
Arguably the most simple approach to solve the inverse problem is just by "trial and error" or hand tuning the parameters. How difficult this can already be for just 3 parameters, we will explore in the following:

### Tasks
- Implement a simulator for a single compartment HH model in `jaxley`
    - For details see the [documentation](https://jaxley.readthedocs.io/en/latest/index.html)
- hand-tune (manually vary) the different conductances $g_{Na}$, $g_{K}$, $g_{leak}$ to fit a single compartment HH model to the measured voltage trace.
- plot the results

### Questions
- How do you assess the quality of the fit, i.e. what (characteristics) do you look for?
- What are ways to quantify this?

_Note: don't forget to set the initial conditions_

In [ ]:
# Set up the simulation

# NOTE: Keep in mind that the stepsize of the solver is 1/10th of the sampling interval of our experimental data!
dt = 0.025

# define the stimulus
stimulus = step_current(
    # add the stimulus parameters here
)
ts = jnp.arange(
    # add the time points here
)

# create a single compartment HH model, add a stimulus and record the voltage

comp = # define the model here 
# add a recording and stimulation here ()

# set the initial conditions
# initialize the model

# define a simulator
@jit
def simulate(params: dict[str, jnp.ndarray]) -> jnp.ndarray:
    """Simulate the HH model for the given parameters.

    Note that we are using `@jit` to compile the function before running it.
    Since we are running the function a lot of times, it is worth the addtional overhead 
    to compile it first.
    
    Args:
        params: The parameters of the HH model.

    Returns:
        v: The voltage trace.
    """
    # set the simulation parameters to update the model
    pstate = None
    for key, value in params.items():
        pstate = comp.data_set(key, value, pstate)

    # integrate the model
    v = jx.integrate(comp, param_state=pstate, delta_t=dt)
    return v[:, :len(ts)] # match length to ts

In [20]:
# define the parameter bounds
lb = {"HH_gNa": 1e-3, "HH_gK": 1e-3, "HH_gLeak": 1e-5}
ub = {"HH_gNa": 1.0, "HH_gK": 1.0, "HH_gLeak": 1e-3}
bounds = {k:(lb,ub) for k, (lb, ub) in zip(lb.keys(), zip(lb.values(), ub.values()))}

In [ ]:
# hand-tune the parameters
param_guess = {"HH_gNa": jnp.array([0.3]), "HH_gK": jnp.array([0.2]), "HH_gLeak": jnp.array([0.0001])}
v_guess = simulate(param_guess)

# plot the voltage trace

## Random / Grid search
While hand-tuning is a very simple method that can work for a few parameters, you should have noticed that this is already tricky for just 3 parameters and it will quickly become infeasible for more complex models and larger parameter spaces. Therefore we will explore a still simple, but more systematic approach: random / grid search.

### Tasks
- Implement a mean absolute error loss
- Implement **either** a random **or** a grid search to find suitable parameters
- Plot your top 10 results as quantified by their mean absolute error

### Questions
- Are the results what you expected? Why / Why not?
- What are different ways in which you could improve the results?

In [22]:
# parallelize the simulation (for more details see https://docs.jax.dev/en/latest/_autosummary/jax.vmap.html)
# this way you can input an entire batch of parameters rather than having to loop over them
parallel_simulate = vmap(simulate)

# define an error / distance function
def mean_abs_err(v_est: jnp.ndarray, v_true: jnp.ndarray) -> jnp.ndarray:
    """Compute the mean absolute error between the estimated and observed voltage trace.
    
    Args:
        v_est: The estimated voltage trace.
        v_true: The true voltage trace.

    Returns:
        err: The mean absolute error.
    """
    return # implement the mean absolute error here

In [23]:
# sample a batch of parameter (randomly or on a grid)

# grid-search
gridsize = (20, 20, 20)

# random search
num_samples = 10_000

In [24]:
# simulate the batch of parameters
v_samples =

# compute the mean absoluteerror
err = mean_abs_err(v_samples, v_obs)

# get the top 10 results
top_indices = 

In [ ]:
# plot the top 10 results and compare them with the observation

## Rejection ABC
While using random and grid searches to sample the parameter space can yield good results, they struggle to deal with uncertainty and noisy data. Bayesian inference tries to alleviate this by infering a posterior distribution, rather than a point estimate. Unfortunately, most simulators have intractable likelihood functions however and hence we can only approximately do Bayesian inference (Approximate Bayesian Computation, ABC). One method to do this is called rejection ABC, where we approximate the posterior using samples from the simulator.

### Tasks
- Implement a rejection ABC sampler using the mean squared error as a distance metric
- Sample the parameter space
- Plot your top 10 parameter samples. Compare them to the previous best samples
- Play around with the threshold to see how it affects the results

### Questions
- Are the results what you expected? Why / Why not?
- What are different ways in which you could improve the results?

_Note you can use the `mean_abs_err` function you defined before._

In [26]:
# implement the rejection ABC sampler
def rejection_abc(num_samples: int, threshold: float, distance_fn: Callable[[jnp.ndarray, jnp.ndarray], jnp.ndarray] = mean_abs_err) -> Tuple[dict[str, jnp.ndarray], jnp.ndarray, jnp.ndarray]:
    """Rejection ABC sampler.
    
    Args:
        num_samples: The number of samples to draw.
        threshold: The threshold for the distance.
        distance_fn: The distance function.
    """
    # Sample parameters from prior
    rng = jax.random.split(jax.random.PRNGKey(0), len(bounds))
    samples = {k: jax.random.uniform(sub_rng, (num_samples,), minval=lb, maxval=ub) 
               for sub_rng, (k, (lb, ub)) in zip(rng, bounds.items())}
    
    # Simulate for all parameter samples
    v_samples = 
    
    # Calculate distances
    distances = 
    
    # Select samples below threshold
    accepted_indices = 
    accepted_params = 
    accepted_distances = 
    
    return accepted_params, accepted_distances, v_samples[accepted_indices]

In [27]:
# Run rejection ABC
num_samples = 10_000
threshold = 13  # You may need to adjust this based on your data
accepted_params, accepted_distances, accepted_v = rejection_abc(num_samples, threshold)

# Sort by distance to get the best fits
sorted_indices = 
top_indices =  # Get top 10

In [ ]:
# visualize the estimated posterior within the prior bounds using `pairplot`
# Note the pairplot expects a torch tensor as input
# you can learn more about `pairplot` if you run `?pairplot`
accepted_samples = torch.tensor(np.array(list(accepted_params.values())).T)
# plot the distribution

In [ ]:
# Plot top voltage traces of the top 10 parameter samples

## Rejection ABC (with features)
Rather than computing the distance on the raw voltage traces of the HH model, it can be beneficial to compute the distance on a lower dimensional representation of the data. This is computationally much easier and more robust. A common approach to reduce the dimensionality of the data is to compute so called summary features, which are able to capture more of the global structure of the data. 

### Tasks
- Implement a distance metric based on summary features
    - How could you quantify global charecteristics of the voltage trace? Think of statistical properties.
    - Come up with your own summary features and implement them
- Run the rejection ABC sampler again with the new distance metric
- Plot your top 10 parameter samples. Compare them to the previous best samples
- Play around with the threshold to see how it affects the results

### Questions
- How do the results compare to the previous results? Which ones would you consider to be better?
- What are ways in which you could improve the results even further?

In [30]:
def compute_summary_stats_np(v: jnp.ndarray) -> jnp.ndarray:
    """Compute summary statistics of the voltage trace.
    
    A feature can be anything that takes the time series as input and returns a single number.

    Args:
        v: The voltage trace.

    Returns:
        summary_stats: The summary statistics.
    """
    ft1 = 
    ft2 = 
    ft3 = 
    ...
    return jnp.hstack([ft1, ft2, ft3, ...])


def fts_err(v_est: jnp.ndarray, v_true: jnp.ndarray) -> jnp.ndarray:
    """Compute a feature based distance.

    ```
    fts_est = compute_summary_stats(v_est)
    fts_true = compute_summary_stats(v_true)

    fts_err = sum(jnp.abs(fts_est - fts_true))
    ```
    
    Args:
        v_est: The estimated voltage trace.
        v_true: The true voltage trace.

    Returns:
        ft_err: The feature based distance.
    """
    # implement the feature based distance here
    return fts_err

In [31]:
# Run rejection ABC
num_samples = 10_000
threshold = 25  # You may need to adjust this based on your data
accepted_params, accepted_distances, accepted_v = rejection_abc(num_samples, threshold, fts_err)

# Sort by distance to get the best fits
sorted_indices = 
top_indices =   # Get top 10

In [ ]:
# visualize the estimated posterior within the prior bounds using `pairplot`
# Note the pairplot expects a torch tensor as input
accepted_samples = torch.tensor(np.array(list(accepted_params.values())).T)
# plot the distribution

In [ ]:
# Plot top 10 simulations

## Neural Posterior Estimation

While rejection ABC provides a more principled approach than grid or random searches, it can become inefficient as the dimensionality of the parameter space increases or when the simulator is computationally expensive. Neural Posterior Estimation (NPE) addresses these limitations by using neural networks to learn an approximation of the posterior distribution directly from simulation data.

NPE leverages the power of conditional density estimation to build a model that maps from observed data to the corresponding posterior distribution over parameters. This approach often requires fewer simulations than traditional ABC methods and can capture complex posterior distributions more effectively.

### Tasks
- Implement NPE using the `sbi` package, use the lower dimensional summary features as input. For details see the [documentation](https://sbi-dev.github.io/sbi/0.22/)
    - Use one round of `SNPE` for that
    - use a MDN as a density estimator (run `?SNPE` for details)
    - reimplement the `compute_summary_stats` function to work with torch tensors
- plot the posterior
- compute the MAP estimate

### Questions
- Under what circumstances might NPE be preferable to rejection ABC, and vice versa?
- How do the posterior estimates from NPE compare to those from rejection ABC in terms of accuracy and efficiency?

_Note: the `sbi` package is based on pytorch and hence expects `torch.tensors` as inputs and outputs. These work similarly to numpy arrays and also come with similar functions, i.e `torch.sum` etc. Use the [documentation](https://sbi-dev.github.io/sbi/0.22/) to answer your questions._

In [ ]:
# define Uniform prior for the parameters
prior = BoxUniform() # use the parameter bounds

# reimplement the summary statistics function to work with torch tensors or 
# wrap the numpy function to work with torch tensors
def compute_summary_stats_torch(x: torch.tensor) -> torch.tensor:
    """Compute summary statistics of the voltage trace.
    
    Args:
        x: The voltage trace.

    Returns:
        summary_stats: The summary statistics.
    """
    return # implement the summary statistics here

# simulator function for sbi
def simulate_for_sbi(theta: torch.tensor) -> torch.tensor:
    """Simulate the HH model for the given parameters.
    
    Args:
        theta: The parameters of the HH model.

    Returns:
        v: The voltage trace.
    """
    # since jaxley expects jax arrays and sbi uses torch tensors
    # we need to convert theta back and forth
    theta = theta.numpy()
    theta = {k:v for k,v in zip(bounds.keys(), theta.T)}
    return torch.tensor(parallel_simulate(theta)).to(torch.float32)

# draw samples from the prior, simulate and compute summary statistics
theta_train = 
v_train = 
x_train = 
x_obs = 

# run NPE (one round of SNPE)
snpe = # set up SNPE
infer = snpe.append_simulations(theta_train, x_train)
estimator = infer.train()
posterior = # build posterior

In [ ]:
# condition the posterior on the observed data
posterior.set_default_x(x_obs)

# sample from the posterior
samples = 

# get the top 10 results and simulate them
top_indices = 
v_top = 

# compute the MAP estimate
theta_map = 
print(f"MAP estimate: gNa={theta_map[:, 0].item():.3f}, gK={theta_map[:, 1].item():.3f}, gLeak={theta_map[:, 2].item():.3f}")

In [ ]:
# visualize the estimated posterior within the prior bounds using `pairplot`and the MAP estimate
# Note the pairplot expects a torch tensor as input

In [ ]:
# plot the top 10 results

## Gradient Descent via Backpropagation of Error
While sampling-based methods like ABC and Neural Posterior Estimation provide distributional estimates of parameters, optimization approaches offer an alternative strategy when we primarily seek point estimates. This can be especially useful when the simulator is computationally expensive but gradients over the simulator parameters are available, since gradient-based optimization usually requires many fewer evaluations of the simulator compared to sampling-based methods.


### Tasks
- Optimize parameters directly on the time series data using gradient descent
- Bonus: Optimize using summary statistics as an alternative approach

### Questions
- How do the results compare to the ones from SBI methods?
- How do the results from the feature based approach compare to the ones from the mean squared error loss?
- What are ways to make the optimization process more robust to local minima?

_Note: to ensure that the parameters are optimized within their bounds, we use use sigmoid to transform between the interval [a,b] <-> [-inf,inf]. You can use the [documentation](https://jaxley.readthedocs.io/en/latest/index.html) and [tutorials](https://jaxley.readthedocs.io/en/latest/tutorials.html) to help you answer the questions._

In [38]:
transform = jx.ParamTransform({k:SigmoidTransform(lb[k], ub[k]) for k in bounds.keys()})

# define the loss function
def loss(params: jnp.ndarray) -> jnp.ndarray:
    """Compute the mean absolute error between the estimated and observed voltage trace.
    
    Args:
        params: The parameters of the HH model.

    Returns:
        loss: The mean absolute error.
    """
    params = transform.forward(params) # transform the parameters
    v_est = 
    err = 
    return err

diff = jit(value_and_grad(loss)) # take the gradient of the loss function (and use jit to speed up the computation)

In [ ]:
# define the optimizer and initialize the parameters (you can play around with the learning rate, number of epochs, etc.)
lr = 1e-1
max_epochs = 5000
init_params = {"HH_gNa": 0.12, "HH_gK": 0.3, "HH_gLeak": 0.0003}
opt_params = transform.inverse(init_params)
optimizer = optax.adam(learning_rate=lr)
opt_state = optimizer.init(opt_params)

# run the optimization

# loop over the epochs
    loss_val, grad_val = # obtain the gradient and value of the loss function

    updates, opt_state = optimizer.update(grad_val, opt_state)
    opt_params = optax.apply_updates(opt_params, updates)
    
# get the final parameters
final_params = transform.forward(opt_params)

In [ ]:
# simulate the final parameters

# plot the results